In [2]:
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

from IPython.display import display, HTML

warnings.filterwarnings('ignore')

c:\Users\exore\anaconda3\envs\kaggle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
btc_data = pd.read_csv('gemini_BTCUSD_2020.csv', sep=',')
btc_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1.618880e+12,4/20/2021 0:02,BTCUSD,55717.47,55723.00,55541.69,55541.69,2.406734
1,1.618880e+12,4/20/2021 0:01,BTCUSD,55768.94,55849.82,55711.74,55717.47,0.573458
2,1.618880e+12,4/20/2021 0:00,BTCUSD,55691.79,55793.15,55691.79,55768.94,3.309904
3,1.618880e+12,4/19/2021 23:59,BTCUSD,55777.86,55777.86,55677.92,55691.79,0.684934
4,1.618880e+12,4/19/2021 23:58,BTCUSD,55803.50,55823.88,55773.08,55777.86,0.477225
...,...,...,...,...,...,...,...,...
658060,1.577840e+12,1/1/2020 0:04,BTCUSD,7163.60,7163.60,7163.00,7163.00,1.751844
658061,1.577840e+12,1/1/2020 0:03,BTCUSD,7163.00,7163.60,7163.00,7163.60,0.002706
658062,1.577840e+12,1/1/2020 0:02,BTCUSD,7163.30,7164.22,7163.00,7163.00,0.063904
658063,1.577840e+12,1/1/2020 0:01,BTCUSD,7163.30,7164.23,7163.30,7163.30,0.002646


=======================PREDICTION WITH FEATURES=======================

In [4]:
btc_data_pr = btc_data.drop(
    columns=[
        'Unix Timestamp', 
        'Date',
        'Symbol',
    ], 
)

scaler = MinMaxScaler()

btc_data_norm = scaler.fit_transform(btc_data_pr)
btc_data_norm

array([[8.49518930e-01, 8.49460489e-01, 8.48311695e-01, 8.46635149e-01,
        2.30622826e-03],
       [8.50363327e-01, 8.51540844e-01, 8.51105153e-01, 8.49518930e-01,
        5.49510071e-04],
       [8.49097633e-01, 8.50611229e-01, 8.50777429e-01, 8.50363327e-01,
        3.17168196e-03],
       ...,
       [5.29575155e-02, 5.29022450e-02, 5.35816634e-02, 5.29525938e-02,
        6.12354704e-05],
       [5.29575155e-02, 5.29024090e-02, 5.35865915e-02, 5.29575155e-02,
        2.53533998e-06],
       [5.30001701e-02, 5.30100192e-02, 5.35865915e-02, 5.29575155e-02,
        7.59975304e-06]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    btc_data_norm[:, :-1], 
    btc_data_norm[:, -1], 
    test_size=0.4, 
    random_state=42
)

print(f"x_train: {len(X_train)}\nx_test:  {len(X_test)}")

x_train: 394839
x_test:  263226


In [6]:
svr = SVR(kernel='rbf', C=100, gamma=0.1)
#svr.fit(X_train, y_train)

In [7]:
y_test_pred = svr.predict(X_test)
print(f"\nMSE is {mean_absolute_error(y_test_pred, y_test)}\n")

NotFittedError: This SVR instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

=======================TIMESERIES PREDICTION=======================

In [21]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence) - n_steps):
        # найти конец шага
        end_ix = i + n_steps
        # разделить последовательность на примеры с входом и выходом
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

X, y = split_sequence(btc_data_norm[:, -1], 200)
print(f"x: {X.shape}")

x: (657865, 200)


In [22]:
svr = SVR(kernel='rbf', C=100, gamma=0.1)
svr.fit(X, y)

: 

In [ ]:
y_test_pred = svr.predict([X[2]])
print(f"\nMSE is {mean_absolute_error([y_test_pred], [y[2]])}\n")


MSE is 0.09656167112545892



In [20]:
svr.predict([X[1]]), y[1]

(array([0.0999476]), 0.000735188281173248)

In [45]:
X[1]

array([0.00054951, 0.00317168, 0.00065633, 0.0004573 , 0.00523363,
       0.00066723, 0.00045053, 0.00043765, 0.00027446, 0.00050238])